<h3>TODO</h3>
<p>El algoritmo consiste principalmente en un <a href=#Loop>loop</a>, en el que para cada archivo:</p>
<ol>
    <p><li><a href=#Leer><b>Leer</b></a>, para tenerlo en un dataframe (<b style="color:green;">OK</b>)</li></p>
    <p><li><a href=#Filtrar><b>Filtrar</b></a>, para quedarnos con los registros y columnas relevantes (<b style="color:green;">OK</b>)</li></p>
    <p><li><a href=#Agregar><b>Agregar</b></a>, datos adicionales, para poder trabajar mejor:
        <ol>
            <li>Agregar mes y año del archivo, para usar en lugar de la fecha de creación (<b style="color:green;">OK</b>)</li>
            <li>Agregar todos los datos que se podrían obtener del campo descripción, como cochera, vista al mar, etc (<b style="color:red;">TODO</b>)</li>
            <li>Agregar todos los datos adicionales resultantes de la unión con los otros dataframes (hospitales, subte, etc) (<b style="color:red;">TODO</b>)</li>
        </ol></li></p>
    <p><li><a href=#Calcular><b>Calcular</b></a>, el valor para todos los registros que no tengan precio: (<b style="color:red;">TODO</b>)
        <ol>
            <li><a href=#Obtener><b>Obtener</b></a> las propiedades más semejantes (como con KNN):
                <ul>
                    <li>Restricción: La distancia temporal entre propiedades no debiera ser mayor a 3-6 meses.</li>
                </ul></li>
            <li>En base a eso, obtener el coeficiente promedio (valor propiedad / (precio unitario promedio * superficie total))</li>
            <li>Finalmente, multiplicar el coeficiente por precio unitario promedio y superficie</li>
        </ol></li></p>
    <p><li><b>Agregar</b>, el dataframe a uno global, para poder predecir mejor las siguientes propiedades, en cuanto a la distancia temporal (<b style="color:red;">TODO</b>)</li></p>
</ol>

<p>Finalmente, se realiza el reporte de las propiedades de las que se quiere obtener el valor (en este caso, agosto 2017) (<b style="color:red;">TODO</b>)</p>

In [ ]:
# No vamos a graficar, de momento
#%matplotlib inline

import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
from os import listdir

In [ ]:
#Funciones de utilería

def SeleccionarSubCampo(campo, separador, indice):
    """Selecciona un elemento de una lista en string"""
    return campo.split(separador)[indice]

def SacarColumna(dataFrame, columnName):
    """Quita una columna de un dataframe. Si no existe, no hace nada"""
    if columnName in dataFrame:
        dataFrame.drop(columnName, axis = 1, inplace = True)

def SacarListaColumnas(dataFrame, columnNameList):
    """Quita una lista de columnas de un dataframe"""
    for columnName in columnNameList:
        SacarColumna(dataFrame, columnName)

def ResolveSerie(serieFrom, function, indexForUse):
    if (serieFrom is None):
        return pd.Series(None, index = indexForUse)
    return pd.Series([function(x) for x in serieFrom], index = indexForUse)

def ResolveColumn(dataFrame, columnNameFrom, columnNameTo, function):
    dataFrame[columnNameTo] = ResolveSerie(dataFrame[columnNameFrom], function, dataFrame.index)

<p>Función que lee dataframes</p>
<a name='Leer'/>

In [ ]:
def LeerDataFrame(nombreArchivo):
    """Función que lee un archivo y devuelve un dataframe"""
    return pd.read_csv(nombreArchivo, low_memory = False)


<p>Funciones que filtran datos</p>
<a name='Filtrar'/>

In [ ]:
def FiltrarPais(dataFrame, listaPaises):
    """Devuelve un dataframe con la lista de países filtrado"""
    if not 'country' in dataFrame:
        ResolveColumn(dataFrame, 'place_with_parent_names', 'country', lambda x: SeleccionarSubCampo(x, '|', 1))
    dataFrame = dataFrame[dataFrame['country'].isin(listaPaises)]
    SacarColumna(dataFrame, 'country')
    return dataFrame

def FiltrarProvincia(dataFrame, listaProvincias):
    """Devuelve un dataframe con la lista de países filtrado"""
    if not 'state_name' in dataFrame:
        ResolveColumn(dataFrame, 'place_with_parent_names', 'state_name', lambda x: SeleccionarSubCampo(x, '|', 2))
    dataFrame = dataFrame[dataFrame['state_name'].isin(listaProvincias)]
    return dataFrame

def EliminarColumnasNoRelevantes(dataSet):
    SacarListaColumnas(dataSet, ['properati_url',
                                 'operation',
                                 'geonames_id',
                                 'lat-lon',
                                 'currency',
                                 'price_aprox_local_currency',
                                 'image_thumbnail'])

def AplicarFiltros(dataFrame):
    dataFrame = FiltrarPais(dataFrame, ['Argentina'])
    dataFrame = FiltrarProvincia(dataFrame, ['Capital Federal',
                               'Bs.As. G.B.A. Zona Norte',
                               'Bs.As. G.B.A. Zona Oeste',
                               'Bs.As. G.B.A. Zona Sur'])
    EliminarColumnasNoRelevantes(dataFrame)
    dataFrame.reindex()
    return dataFrame


In [ ]:
def ProcesarDescripcion(descripcion):
    diccionario = {}
    return diccionario

def ProcesarSerieDescripcion(dataFrame):
    return ResolveSerie(dataFrame['descripcion'] if 'A' in dataFrame.columns else None, ProcesarDescripcion, dataFrame.index)

<p>Funciones que agregan información<p>
<a name='Agregar'/>

In [ ]:
def AgregarFechaDeArchivo(dataFrame, archivo):
    archiveParts = archivo.split('-')
    dumpDate = archiveParts[2] + '-' + archiveParts[3]
    dataFrame['dump_date'] = pd.Series(dumpDate, index = dataFrame.index)

def AgregarBarrio(dataFrame):
    if not 'barrio' in dataFrame:
        ResolveColumn(dataFrame, 'place_with_parent_names', 'barrio', lambda x: SeleccionarSubCampo(x, '|', 3))
    SacarColumna(dataFrame, 'place_with_parent_names')

def AgregarCampoDesdeDescripcion(descripcionesProcesadas, campo, dataFrame):
    dataFrame[campo] = ResolveSerie(descripcionesProcesadas, lambda descripcion: descripcion[campo], dataFrame.index)

def AgregarOtrosDatos(dataFrame):
    descripcionesProcesadas = ProcesarSerieDescripcion(dataFrame)
    #como ejemplo:
    #AgregarCampoDesdeDescripcion(descripcionesProcesadas, 'cochera', dataFrame)

def AgregarInformacion(dataFrame, archivo):
    AgregarFechaDeArchivo(dataFrame, archivo)
    AgregarBarrio(dataFrame)
    AgregarOtrosDatos(dataFrame)
    return dataFrame

<p>Obtener los vecinos más semejantes</p>
<a name='Obtener'/>

In [ ]:

def ObtenerVecinosCluster(registro, criterio):
    """Devuelve los integrantes de un cluster, según algún criterio de clusterización."""
    listaVecinos = []
    #TODO: Implementar la función. Vas a tener que crear las clusterizaciones antes, viejo
    return listaVecinos

def ObtenerListaDeVecinosCluster(registro, listaCriterios):
    lista_de_listaVecinos = []
    for criterio in listaCriterios:
        lista_de_listaVecinos += ObtenerVecinosCluster(registro, listaCriterios)

    return lista_de_listaVecinos

def InnerJoinDeVecinos(listas_de_listaVecinos, lista_funciones_que_aplican_criterios_para_figurar):
    """Devuelve una lista de vecinos que aparezcan en varias listas, según algunos criterios"""
    listaDeVecinos_versionFinal_9999 = []
    #TODO: Implementar la función. Si, ya se que todo este quilombo sería más fácil en Spark,
    # pero no como no anda en windows vas a tener que arreglártelas solo
    return listaDeVecinos_versionFinal_9999

def ObtenerListaPosiblesVecinos(registro):
    """Devuelve una lista con los candidatos a vecinos más cercanos"""
    listaPosiblesVecinos = []
    #TODO: Implementar la función.
    #Tener en cuenta que es cara. Buscar maneras de optimizar (clusterización multicapa?)

    listaCriterios = [] #todos los tipos de criterios con los que se clusterizó
    lista_de_listaVecinos = ObtenerListaDeVecinosCluster(registro, listaCriterios)

    lista_funciones_que_aplican_criterios_para_figurar = [] #andá a saber cuáles son
    listaDeCandidatosAVecinos = InnerJoinDeVecinos(lista_de_listaVecinos, lista_funciones_que_aplican_criterios_para_figurar)
    return listaDeCandidatosAVecinos

def ObtenerKVecinosMasCercanos(registro, k):
    """Devuelve una lista de los k vecinos más cercanos a una propiedad, siendo cada elemento
    una tupla del coeficiente general y su semejanza con el registro en cuestión"""
    vecinos = [] # algún heap de máximos, donde la semejanza sea el comparable y el registroVecino el dato
    #TODO: Implementar la función (lo más eficiente posible, pleaseeee)
    listaDeCandidatosAVecinos = ObtenerListaPosiblesVecinos(registro)

    for posibleVecino in listaDeCandidatosAVecinos:
        relevancia = False #que_se_yo(registro, posibleVecino, otros_datos)
        if relevancia:
            vecinos += posibleVecino

    #sacar los k primeros (más semejantes)
    return vecinos[:k]


<p>Calcular el precio de una propiedad</p>
<a name='Calcular'/>

In [ ]:
#TODO: Implementar, completar, refactorizar, ya tu sabes

def ObtenerPrecioUnitarioPromedio(barrio, año_y_mes):
    """Devuelve el precio unitario promedio de un barrio. En caso de no existir, devuelve 0"""
    #TODO: Implementar la función
    #if no_hay_data(barrio, año_y_mes):
        #return 0
    return 1000

def ObtenerListaDePreciosUnitarios(barrio, año_y_mes, k_meses_regresion):
    """Devuelve una lista de valores para realizar una regresión"""
    listaValores = []
    #TODO: Implementar la función
    return listaValores


def CalcularRegresion(listaValores):
    #TODO: Mirar regresion en apuntes de Analisis Numerico y completar
    valor = 0
    return valor

def CalcularPrecioUnitarioPromedioPorRegresion(barrio, año_y_mes):
    """Calcula por regresión el precio unitario promedio de un barrio"""    
    #TODO: Implementar la función. No usar más de 3-6 meses.
    #En serio, cambiar por un número
    k_meses_regresion = np.random.random_integers(low = 3, high = 6)

    lista_datos_regresion = ObtenerListaDePreciosUnitarios(barrio, año_y_mes, k_meses_regresion)
    precioUnitario = CalcularRegresion(lista_datos_regresion)
    return precioUnitario

def CalcularPrecioPorLimitroficidadDeBarrios(barrio, año_y_mes):
    """Calcula el precio unitario promedio de un barrio según los limitrofes"""
    #TODO: Implementar la funcion. Tener cuidado de no caer en loops infinitos.
    return 0

def CalcularPrecioUnitarioPromedio(barrio, año_y_mes):
    """Calcula precio unitario promedio mediante regresión. En caso de que no halla
    registros para ese barrio, tomar el promedio de los barrios limítrofes"""
    precioUnitario = CalcularPrecioUnitarioPromedioPorRegresion(barrio, año_y_mes)
    if (precioUnitario == 0):
        CalcularPrecioPorLimitroficidadDeBarrios(barrio, año_y_mes)
    return precioUnitario

def CalcularPrecioUnitarioPromedioGeneral(barrio, año_y_mes):
    """Devuelve el precio unitario promedio según el barrio y el año/mes.
    En caso de no figurar, se calcula por regresión y/o limitroficidad de barrios"""    
    precioUnitario = ObtenerPrecioUnitarioPromedio(barrio, año_y_mes)
    if (precioUnitario == 0):
        precioUnitario = CalcularPrecioUnitarioPromedio(barrio, año_y_mes)    
    return precioUnitario

def ObtenerPrecioUnitarioPromedio(registro):
    """Devuelve el precio unitario promedio según el barrio y el año/mes de una publicación de venta"""
    return CalcularPrecioUnitarioPromedioGeneral(registro['barrio'], registro['dump_date'])

def ObtenerCoeficienteGeneral(registro):
    """Devuelve el coeficiente general de un registro"""
    return registro['price'] / (registro['surface_in_m2'] * ObtenerPrecioUnitarioPromedio(registro))

def CalcularCoeficienteGeneral(registro):
    """Calcula el coeficiente general en base a un promedio ponderado con propiedades semejantes.
    Para esto se requiere obtener un listado de los k vecinos más semejantes (símil KNN)"""
    #TODO: determinar k? usar clusterización? etc?
    k = np.random.random_integers(low = 5, high = 100)
    listaVecinos = ObtenerKVecinosMasCercanos(registro, k)
    coeficiente = 0
    for vecino in listaVecinos:
        coeficiente += vecino[0] * vecino[1]
    return coeficiente

def CalcularPrecio(registro):
    precioUnitarioPromedio = ObtenerPrecioUnitarioPromedio(registro)
    coeficienteGeneral = CalcularCoeficienteGeneral(registro)
    return registro['surface_in_m2'] * coeficienteGeneral * precioUnitarioPromedio

In [ ]:
#Funciones útiles

def CrearDataFrame(columnas):
    """Crea un dataframe vacío, cuyas columnas se crean a
    partir de una lista de pares nombre y dtype"""
    dataframe = pd.DataFrame()
    for col in columnas_gBarriosPrecios:
        dataframe[col[0]] = pd.Series(name = col[0], dtype = col[1])
    return dataframe


In [ ]:
#variables "globales"

# Ruta de la carpeta con los archivos de datos originales
gRutaCarpetaOriginales = "./properties/"

#TODO: revisar si vale la pena
# Dataframe con estadísticas de precio por barrio y mes

columnas_gBarriosPrecios = [('state_name', np.dtype('str')),
                            ('place_name', np.dtype('str')),
                            ('entry_date', np.dtype('str')),
                            ('mean price', np.dtype('float')),
                            ('min_price', np.dtype('float')),
                            ('max_price', np.dtype('float'))]

gBarriosPrecios = CrearDataFrame(columnas_gBarriosPrecios)

gBarriosPrecios.info()

In [ ]:
# Hacemos un rápido chequeo de los archivos, para saber si alguno se descargó mal
def CheckCorruptDownloads():
    for archive in listdir(gRutaCarpetaOriginales):
        if ".csv" in archive:
            nombreArchivo = gRutaCarpetaOriginales + archive
            try:
                print(nombreArchivo, end = ': ')
                df = pd.read_csv(nombreArchivo, low_memory = False)
                print('Ok')
            except ValueError:
                print("Error, archivo corrupto. Descargar de nuevo")

#CheckCorruptDownloads()

In [ ]:
def ProcesarDataFrame(rutaArchivo, nombreArchivo):
    """Procesa un archivo csv, y devuelve un dataFrame listo para usar"""
    dataFrame = LeerDataFrame(rutaArchivo)
    dataFrame = AplicarFiltros(dataFrame)
    dataFrame = AgregarInformacion(dataFrame, nombreArchivo)
    return dataFrame


<p>Loop principal</p>
<a name='Loop'/>

In [ ]:
for archive in listdir(gRutaCarpetaOriginales):
    if ".csv" in archive:
        print(archive)
        df = ProcesarDataFrame(gRutaCarpetaOriginales + archive, archive)

        if 'price' not in df:
            df['price'] = pd.Series(name = 'price', dtype = np.dtype('float'))

        #TODO: Calcular precio en caso de ser nulo o cero

        print(df.info())
    break